# DREAM advanced data reduction

- Audience: Instrument (data) scientists, instrument users
- Prerequisites: Basic knowledge of [Scipp](https://scipp.github.io/), [Sciline](https://scipp.github.io/sciline/)

This notebook builds on the [basic powder workflow](./dream-powder-reduction.rst) and demonstrates how the workflow can be used to compute different results and how alternative steps can be used.

This notebook uses the same data as the basic notebook, a McStas + GEANT4 simulation.
The data is available through the ESSdiffraction package but accessing it requires the `pooch` package.
If you get an error about a missing module `pooch`, you can install it with `!pip install pooch`:

In [ ]:
import scipp as sc
from ess import dream, powder
import ess.dream.data  # noqa: F401
from ess.powder.types import *
import pandas as pd
import plopp as pp

## Compute intensity as a function of scattering angle

The basic notebook sums over all detector voxels and produces a 1D curve.
Here, we instead bin by scattering angle $2\theta$.

First, define the same workflow as in the [basic example](./dream-powder-reduction.rst#create_and_configure_the_workfow):

In [ ]:
workflow = dream.DreamGeant4Workflow(run_norm=powder.RunNormalization.monitor_histogram)

workflow[Filename[SampleRun]] = dream.data.simulated_diamond_sample()
workflow[Filename[VanadiumRun]] = dream.data.simulated_vanadium_sample()
workflow[Filename[BackgroundRun]] = dream.data.simulated_empty_can()
workflow[CalibrationFilename] = None

workflow[MonitorFilename[SampleRun]] = dream.data.simulated_monitor_diamond_sample()
workflow[MonitorFilename[VanadiumRun]] = dream.data.simulated_monitor_vanadium_sample()
workflow[MonitorFilename[BackgroundRun]] = dream.data.simulated_monitor_empty_can()
workflow[CaveMonitorPosition] = sc.vector([0.0, 0.0, -4220.0], unit="mm")

workflow[dream.InstrumentConfiguration] = dream.InstrumentConfiguration.high_flux
# Select a detector bank:
workflow[NeXusDetectorName] = "mantle"
# We drop uncertainties where they would otherwise lead to correlations:
workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.drop
# Edges for binning in d-spacing:
workflow[DspacingBins] = sc.linspace("dspacing", 0.3, 2.3434, 201, unit="angstrom")

# Do not mask any pixels / voxels:
workflow = powder.with_pixel_mask_filenames(workflow, [])

And then add the desired bin edges for $2\theta$:

In [ ]:
workflow[TwoThetaBins] = sc.linspace(
    dim="two_theta", unit="rad", start=0.8, stop=2.4, num=201
)

Instead of computing `IofDspacing` and from that `IofTof` as in the basic example, here, we want to compute `IofDspacingTwoTheta`:

In [ ]:
workflow.visualize(IofDspacingTwoTheta, graph_attr={"rankdir": "LR"})

Now we can compute the intensity as a function of $2\theta$ and $d$-spacing by requesting `IofDspacingTwoTheta`:

In [ ]:
grouped_dspacing = workflow.compute(IofDspacingTwoTheta)
grouped_dspacing

In [ ]:
grouped_dspacing.hist().plot(title=grouped_dspacing.coords['detector'].value.capitalize(),
                             norm="log")

## Alternative run normalizations

The [basic example](./dream-powder-reduction.rst) normalizes the detector data by a monitor that was histogrammed in wavelength.
ESSdiffraction provides some alternatives.

### Normalize by integrated monitor

Instead of computing a histogram of the monitor data, we can integrate over all bins to get a single intensity value for the monitor.

To do so, specify `ess.powder.RunNormalization.monitor_integrated` when constructing the workflow.
This will insert [normalize_by_monitor_integrated](../../generated/modules/ess.powder.correction.normalize_by_monitor_integrated.rst) into the workflow.
Then, set parameter as before.

In [ ]:
workflow = dream.DreamGeant4Workflow(run_norm=powder.RunNormalization.monitor_integrated)

workflow[Filename[SampleRun]] = dream.data.simulated_diamond_sample()
workflow[Filename[VanadiumRun]] = dream.data.simulated_vanadium_sample()
workflow[Filename[BackgroundRun]] = dream.data.simulated_empty_can()
workflow[CalibrationFilename] = None

workflow[MonitorFilename[SampleRun]] = dream.data.simulated_monitor_diamond_sample()
workflow[MonitorFilename[VanadiumRun]] = dream.data.simulated_monitor_vanadium_sample()
workflow[MonitorFilename[BackgroundRun]] = dream.data.simulated_monitor_empty_can()
workflow[CaveMonitorPosition] = sc.vector([0.0, 0.0, -4220.0], unit="mm")

workflow[dream.InstrumentConfiguration] = dream.InstrumentConfiguration.high_flux
# Select a detector bank:
workflow[NeXusDetectorName] = "mantle"
# We drop uncertainties where they would otherwise lead to correlations:
workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.drop
# Edges for binning in d-spacing:
workflow[DspacingBins] = sc.linspace("dspacing", 0.3, 2.3434, 201, unit="angstrom")

# Do not mask any pixels / voxels:
workflow = powder.with_pixel_mask_filenames(workflow, [])

Looking at the graph, we can see that this only differs from the histogrammed monitor normalization in how `NormalizedRunData` is computed:

In [ ]:
workflow.visualize(IofDspacing, graph_attr={"rankdir": "LR"})

And compute the result:

In [ ]:
result = workflow.compute(IofDspacing)
result.hist().plot(title=result.coords['detector'].value.capitalize())

### Normalize by proton charge

We can normalize the detector data by the accumulated proton charge.
This works similarly to normalizing by a monitor, but we pass `ess.powder.RunNormalization.proton_charge` when building the workflow.
This will insert [normalize_by_proton_charge](../../generated/modules/ess.powder.correction.normalize_by_proton_charge.rst) into the workflow.

In [ ]:
workflow = dream.DreamGeant4Workflow(run_norm=powder.RunNormalization.proton_charge)

workflow[Filename[SampleRun]] = dream.data.simulated_diamond_sample()
workflow[Filename[VanadiumRun]] = dream.data.simulated_vanadium_sample()
workflow[Filename[BackgroundRun]] = dream.data.simulated_empty_can()
workflow[CalibrationFilename] = None

workflow[dream.InstrumentConfiguration] = dream.InstrumentConfiguration.high_flux
# Select a detector bank:
workflow[NeXusDetectorName] = "mantle"
# We drop uncertainties where they would otherwise lead to correlations:
workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.drop
# Edges for binning in d-spacing:
workflow[DspacingBins] = sc.linspace("dspacing", 0.3, 2.3434, 201, unit="angstrom")

# Do not mask any pixels / voxels:
workflow = powder.with_pixel_mask_filenames(workflow, [])

Looking at the graph, we can see that this differs from the monitor normalizations in how `NormalizedRunData` is computed.
And since we don't need them here, the monitor providers and parameters are not in the graph:

In [ ]:
workflow.visualize(IofDspacing, graph_attr={"rankdir": "LR"})

And compute the result as normal:

In [ ]:
result = workflow.compute(IofDspacing)
result.hist().plot(title=result.coords['detector'].value.capitalize())

## Compute intermediate results

For inspection and debugging purposes, we can also compute intermediate results.
To avoid repeated computation (including costly loading of files), we can request multiple results at once, including the final result, if desired.
For example:

In [ ]:
intermediates = workflow.compute(
    (
        DataWithScatteringCoordinates[SampleRun],
        MaskedData[SampleRun],
    )
)

intermediates[DataWithScatteringCoordinates[SampleRun]]

In [ ]:
two_theta = sc.linspace("two_theta", 0.8, 2.4, 301, unit="rad")
intermediates[MaskedData[SampleRun]].hist(
    two_theta=two_theta, wavelength=300
).plot(norm="log")

## Process all detector banks

The other sections only use a single detector bank.
In practice, we want to process all banks.
This section demonstrates how to do this, except for the sans detector which requires a different workflow.

We construct the workflow as before but this time **without specifying a detector name**:

In [ ]:
workflow = dream.DreamGeant4Workflow(run_norm=powder.RunNormalization.proton_charge)

workflow[Filename[SampleRun]] = dream.data.simulated_diamond_sample()
workflow[Filename[VanadiumRun]] = dream.data.simulated_vanadium_sample()
workflow[Filename[BackgroundRun]] = dream.data.simulated_empty_can()
workflow[CalibrationFilename] = None

workflow[dream.InstrumentConfiguration] = dream.InstrumentConfiguration.high_flux
# We drop uncertainties where they would otherwise lead to correlations:
workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.drop
# Edges for binning in d-spacing:
workflow[DspacingBins] = sc.linspace("dspacing", 0.3, 2.3434, 201, unit="angstrom")

# Do not mask any pixels / voxels:
workflow = powder.with_pixel_mask_filenames(workflow, [])

At this point, the workflow is incomplete because it is missing the `NexusDetectorName` parameter.
This will be fixed below.

### 1D intensity vs. ToF

Now, we build a parameter table (as a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html)) with the desired detector names.
(Note that we could use a simple `dict` as well if Pandas was not available.)
Then we [map](https://scipp.github.io/sciline/user-guide/parameter-tables.html) the workflow over those detector names to apply the workflow to each bank separately.
We could do this at some intermediate step, but it is easiest to map the final result.
Finally, we combine the data arrays for the individual detectors into a single data group.

In [ ]:
detector_names = ["mantle", "endcap_forward", "endcap_backward", "high_resolution"]
parameter_table = pd.DataFrame(
    {NeXusDetectorName: detector_names},
    index=detector_names
).rename_axis(index='detector')

all_detector_workflow = workflow.copy()
mapped = all_detector_workflow[IofDspacing].map(parameter_table)
all_detector_workflow[IofDspacing] = mapped.reduce(func=powder.grouping.collect_detectors)

The graph shows all domain types that are used separately for each detector bank as a box instead of a flat square:

In [ ]:
all_detector_workflow.visualize(IofDspacing, graph_attr={"rankdir": "LR"}, compact=True)

Now compute the result:

In [ ]:
result = all_detector_workflow.compute(IofDspacing)

In [ ]:
result

We can plot the detectors individually.
(The range covered by the $d$-spacing bins chosen above is too wide for some banks.
This leads to some bins containing NaN or INF, which is masked out by the workflow.
Here, we set those values to 0 to improve the plot.)

In [ ]:
histograms = result.hist()
for h in histograms.values():
    if 'zero_vanadium' in h.masks:
        h.values[h.masks['zero_vanadium'].values] = 0
histograms.plot()

### 2D intensity vs. d-spacing and 2θ

Just like before, we map the workflow over the detector names.
But here we also assign different $2\theta$ bins for each detector bank.
Those bins are chosen arbitrarily here such that they don't overlap.
This will simplify later steps.

In [ ]:
detector_names = ["mantle", "endcap_forward", "endcap_backward", "high_resolution"]
two_theta_bins = [
    sc.linspace(dim="two_theta", unit="rad", start=0.77, stop=2.36, num=201),
    sc.linspace(dim="two_theta", unit="rad", start=0.24, stop=0.71, num=101),
    sc.linspace(dim="two_theta", unit="rad", start=2.42, stop=2.91, num=151),
    sc.linspace(dim="two_theta", unit="rad", start=2.91, stop=3.11, num=51),
]
parameter_table = pd.DataFrame(
    {NeXusDetectorName: detector_names,
     TwoThetaBins: two_theta_bins,
     },
    index=detector_names
).rename_axis(index='detector')

all_detector_workflow = workflow.copy()
mapped = all_detector_workflow[IofDspacingTwoTheta].map(parameter_table)
all_detector_workflow[IofDspacingTwoTheta] = mapped.reduce(func=powder.grouping.collect_detectors)

In [ ]:
all_detector_workflow.visualize(IofDspacing, graph_attr={"rankdir": "LR"}, compact=True)

In [ ]:
result = all_detector_workflow.compute(IofDspacingTwoTheta)

We can 'sum' over $2\theta$ by using `concat` to obtain a 1D curve:

In [ ]:
sum(da.bins.concat('two_theta').hist() for da in result.values()).plot()

But we can also preserve the $2\theta$ dimension to produce a 2D plot.
Since the detector banks have different, non-overlapping, $2\theta$ ranges, we can simply plot all data arrays into a single figure:

In [ ]:
pp.imagefigure(*(pp.Node(da.hist()) for da in result.values()), norm='log', cbar=True)